<a href="https://colab.research.google.com/github/Baekhyeonjeong/movie-review-/blob/main/%EC%98%81%ED%99%94_%EB%B3%B4%ED%97%A4%EB%AF%B8%EC%95%88%EB%9E%A9%EC%86%8C%EB%94%94_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [69]:
# konlpy 설치하기
!pip install konlpy

In [70]:
# konlpy를 실행하기
import konlpy

In [71]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [72]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [73]:
## 문서 ID로 실제 파일 불러오기
# 보헤미안.csv
## https://drive.google.com/file/d/1wnCxf41BAzW7C-_h0-MNilM7-Y0c05Wy/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1wnCxf41BAzW7C-_h0-MNilM7-Y0c05Wy'})
rawdata_downloaded.GetContentFile('보헤미안.csv')

In [74]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('보헤미안.csv', sep=",")

In [75]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,림키(kkl1****),살아생전 못가본 퀸 콘서트에 갔다왔습니다.,10,13155,418,2018-10-31 11:35:00,0
1,1,조용일(whdy****),브라이언메이 싱크로율 말이되냐,10,8806,319,2018-10-31 11:29:00,0
2,2,Bang JongHo(shom****),슬픈장면은 없는데 눈물 나는 영화 ㅠㅜ,10,7569,231,2018-10-31 13:39:00,0
3,3,최와와(choi****),그 시절 영국에는 두명의 여왕이 있었다,9,7133,351,2018-10-31 09:07:00,0
4,4,김석환(kims****),브라이언 메이는 본인을 데려오신건가요?,10,5833,174,2018-10-31 13:37:00,0
...,...,...,...,...,...,...,...,...
38775,38775,cown****,아무리봐도 9점대는 아니고 7점대 영화라서 1점 줌,1,3,32,2018-12-03 22:27:00,0
38776,38776,끌로이홍(chlo****),동성애는 죄악입니다. 회개하고 하나님 품으로 돌아오세요. 청년들이 이런 영화보고 그...,1,3,32,2018-11-10 22:02:00,0
38777,38777,미쓰백(hhs2****),미쓰백이 10월 전체 평점 1위였는데 갑자기 마지막날 이게 평점 1위된건지 황당하다...,1,4,33,2018-10-31 23:34:00,0
38778,38778,초경합금(toor****),평점조작에 속지마세요 mx관에서 봤다가 돈아까워서 눈물나옵니다.,1,5,38,2018-11-04 12:59:00,0


In [76]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [77]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,림키(kkl1****),살아생전 못가본 퀸 콘서트에 갔다왔습니다.,10,13155,418,2018-10-31 11:35:00,0
1,1,조용일(whdy****),브라이언메이 싱크로율 말이되냐,10,8806,319,2018-10-31 11:29:00,0
2,2,Bang JongHo(shom****),슬픈장면은 없는데 눈물 나는 영화 ㅠㅜ,10,7569,231,2018-10-31 13:39:00,0
3,3,최와와(choi****),그 시절 영국에는 두명의 여왕이 있었다,9,7133,351,2018-10-31 09:07:00,0
4,4,김석환(kims****),브라이언 메이는 본인을 데려오신건가요?,10,5833,174,2018-10-31 13:37:00,0
...,...,...,...,...,...,...,...,...
38775,38775,cown****,아무리봐도 9점대는 아니고 7점대 영화라서 1점 줌,1,3,32,2018-12-03 22:27:00,0
38776,38776,끌로이홍(chlo****),동성애는 죄악입니다. 회개하고 하나님 품으로 돌아오세요. 청년들이 이런 영화보고 그...,1,3,32,2018-11-10 22:02:00,0
38777,38777,미쓰백(hhs2****),미쓰백이 10월 전체 평점 1위였는데 갑자기 마지막날 이게 평점 1위된건지 황당하다...,1,4,33,2018-10-31 23:34:00,0
38778,38778,초경합금(toor****),평점조작에 속지마세요 mx관에서 봤다가 돈아까워서 눈물나옵니다.,1,5,38,2018-11-04 12:59:00,0


# 형태소 분석

In [78]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [79]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [80]:
# "형태소" 변수의 저장 내용 확인
형태소

[('절대', 'Noun'),
 ('보지마', 'Verb'),
 ('.', 'Punctuation'),
 ('왕', 'Noun'),
 ('짜증', 'Noun'),
 ('.', 'Punctuation'),
 ('보다보다', 'Verb'),
 ('중건', 'Noun'),
 ('나옴', 'Verb'),
 ('.', 'Punctuation'),
 ('좋다는', 'Adjective'),
 ('의견', 'Noun'),
 ('은', 'Josa'),
 ('아마', 'Noun'),
 ('돈빋', 'Noun'),
 ('고', 'Josa'),
 ('알바생', 'Noun'),
 ('들', 'Suffix'),
 ('이', 'Josa'),
 ('댓글달았을듯', 'Verb'),
 ('.', 'Punctuation'),
 ('수입', 'Noun'),
 ('했으니', 'Verb'),
 ('본전', 'Noun'),
 ('은', 'Josa'),
 ('뽑아', 'Verb'),
 ('야할테고', 'Adjective'),
 ('.', 'Punctuation'),
 ('한국영', 'Noun'),
 ('화가', 'Noun'),
 ('최고다', 'Noun'),
 ('.', 'Punctuation'),
 ('.', 'Punctuation'),
 ('살다', 'Verb'),
 ('살다', 'Verb'),
 ('이렇게', 'Adverb'),
 ('재미없는', 'Adjective'),
 ('ㆍ', 'Foreign'),
 ('영화', 'Noun'),
 ('는', 'Josa'),
 ('.', 'Punctuation')]

In [81]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,절대,Noun,2,1
1,보지마,Verb,2,1
2,.,Punctuation,2,1
3,왕,Noun,2,1
4,짜증,Noun,2,1
5,.,Punctuation,2,1
6,보다보다,Verb,2,1
7,중건,Noun,2,1
8,나옴,Verb,2,1
9,.,Punctuation,2,1


In [82]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[      0            1  type  count
 0  살아생전         Noun    10      1
 1     못   VerbPrefix    10      1
 2    가본         Verb    10      1
 3     퀸         Noun    10      1
 4   콘서트         Noun    10      1
 5     에         Josa    10      1
 6    갔다         Verb    10      1
 7  왔습니다         Verb    10      1
 8     .  Punctuation    10      1,       0     1  type  count
 0  브라이언  Noun    10      1
 1    메이  Noun    10      1
 2   싱크로  Noun    10      1
 3     율  Noun    10      1
 4     말  Noun    10      1
 5     이  Josa    10      1
 6    되냐  Verb    10      1,      0               1  type  count
 0   슬픈       Adjective    10      1
 1   장면            Noun    10      1
 2    은            Josa    10      1
 3  없는데       Adjective    10      1
 4   눈물            Noun    10      1
 5    나            Noun    10      1
 6    는            Josa    10      1
 7   영화            Noun    10      1
 8   ㅠㅜ  KoreanParticle    10      1,      0           1  type  count
 0    그        Noun    

In [84]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [85]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,살아생전,Noun,10,1
1,못,VerbPrefix,10,1
2,가본,Verb,10,1
3,퀸,Noun,10,1
4,콘서트,Noun,10,1
...,...,...,...,...
36,재미없는,Adjective,2,1
37,ㆍ,Foreign,2,1
38,영화,Noun,2,1
39,는,Josa,2,1


In [86]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [87]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,살아생전,Noun,10,1
1,못,VerbPrefix,10,1
2,가본,Verb,10,1
3,퀸,Noun,10,1
4,콘서트,Noun,10,1
...,...,...,...,...
36,재미없는,Adjective,2,1
37,ㆍ,Foreign,2,1
38,영화,Noun,2,1
39,는,Josa,2,1


In [88]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [89]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소  품사           분류
    Foreign      9     1
!    Foreign      10    4
     Punctuation  1     9
                  2     3
                  3     1
                       ..
！！   Foreign      10    1
！！！  Foreign      10    2
／／   Foreign      10    1
＼＼   Foreign      10    1
￦    Foreign      10    1
Name: 카운트, Length: 52971, dtype: int64

In [90]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [91]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소 품사          분류     
   Foreign     9     1
!   Foreign     10    4
    Punctuation 1     9
                2     3
                3     1
...                 ...
！！  Foreign     10    1
！！！ Foreign     10    2
／／  Foreign     10    1
＼＼  Foreign     10    1
￦   Foreign     10    1

[52971 rows x 1 columns]

In [92]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')